# Lukai - Least Squares Example Workbook

First we need to install the Luk.ai management library and import the dependencies.

In [28]:
!pip install --upgrade -q lukai

In [29]:
import tensorflow as tf
import numpy as np
import lukai

Time to actually create the model! This is a simple least squared model that tries to find the best solution to `y = w * x + b`.

In [30]:
x = tf.placeholder(tf.float32, [None, 1], name="x")
y_ = tf.placeholder(tf.float32, [None, 1], name="y_")

b = tf.Variable(tf.zeros([1]), name="b")
w = tf.Variable(tf.zeros([1, 1]), name="w")

y = tf.identity(w * x + b, name="y")

loss = tf.reduce_sum((y - y_) * (y - y_))
learning_rate = 0.005
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, name="train")

sess = tf.Session()
sess.run(tf.initialize_all_variables())

If you were training this model in Python you'd run something like this, but since we're training it through Luk.ai there's no need.

```python
for step in range(10):
    sess.run(train_step, feed_dict={x:[[2.3],[1.7],[-3.8],[0.5],[-4.1],[-1.5],[-2.5],[6.2]],
                                    y_:[[-4.4],[-3.6],[7.7],[-0.9],[8.3],[2.9],[4.9],[-12.2]]})
    print step, sess.run(w), sess.run(b)
```

Next we print all the tensorflow targets and placeholder names. We'll need these in the client libraries in order to actually feed the model data and run the training targets.

In [32]:
print('Node names: x = {}, y_ = {}, train_step = {}, w = {}, b = {}, y = {}'.format(
  x.name, y_.name, train_step.name, w.name, b.name, y.name,
))

Node names: x = x_2:0, y_ = y__2:0, train_step = train_2, w = w_2:0, b = b_2:0, y = y_2:0


The last step it to upload the model to Luk.ai. You'll need to insert your API token and domain name. If you created a `model_type` other than "leastsquares" you'll need to change that as well.

In [0]:
lukai.set_api_token('<your api token>') # change this to your API token
lukai.upload(
    session=sess,
    domain='<your domain>', # change this to your domain
    model_type='leastsquares', # might have to change this
    name='Hello Model!',
    description='This is a basic least squares model used for beginners.',
    hyper_params=lukai.HyperParams(
        num_clients = 10,
        batch_size = 10,
        num_rounds = 100,
        learning_rate = learning_rate,
        num_local_rounds = 10,
    ),
    metrics={
      loss: lukai.REDUCE_MEAN,
    },
)